# T Structure Function

In [44]:
# imports
from importlib import reload
import os
import glob

import numpy as np

from scipy.io import loadmat

from cugn import gliderdata
from cugn import gliderpairs
from cugn import utils as cugn_utils

# Load up

        time: [85x1 double]
         lat: [85x1 double]
         lon: [85x1 double]
       timeu: [85x1 double]
        latu: [85x1 double]
        lonu: [85x1 double]
           u: [85x1 double]
           v: [85x1 double]
       depth: [100x1 double]
           t: [100x85 double]
           s: [100x85 double]
          fl: [100x85 double]
       theta: [100x85 double]
       sigma: [100x85 double]
         rho: [100x85 double]
    function: 'binsat'
     bintime: 1.7396e+09
        udop: [100x85 double]
        vdop: [100x85 double]
         abs: [100x85 double]

In [34]:
dataset = 'ARCTERX-Leg2'
datafile = '/home/xavier/Projects/Oceanography/data/Spray/ARCTERX/Leg2/0033.mat'
reload(gliderdata)
s33 = gliderdata.FieldData(datafile, dataset)
s33

FieldData object for ARCTERX-Leg2
  Number of profiles: 61
  Time range: 1738456185.0000043 to 1739552955.0000005
  Variables:
    depth: (100,)
    lat: (61,)
    lon: (61,)
    time: (61,)
    missid: (61,)
    dist: (61,)
    s: (100, 61)
    t: (100, 61)
    fl: (100, 61)
    theta: (100, 61)

In [35]:
s33.theta.shape

(100, 61)

# Combine Gliders

In [40]:
reload(gliderdata)
reload(cugn_utils)
datafiles = glob.glob('/home/xavier/Projects/Oceanography/data/Spray/ARCTERX/Leg2/*.mat')
datafiles.sort()
arcterx2 = gliderdata.FieldData.from_list(datafiles, dataset)
arcterx2

FieldData object for ARCTERX-Leg2
  Number of profiles: 180
  Time range: 1738456185.0000043 to 1739560514.9999974
  Variables:
    depth: (100,)
    lat: (180,)
    lon: (180,)
    time: (180,)
    missid: (180,)
    dist: (180,)
    offset: (180,)
    s: (100, 180)
    t: (100, 180)
    fl: (100, 180)
    theta: (100, 180)

In [41]:
arcterx2.dist.shape, arcterx2.offset.shape

((180,), (180,))

# Generate pairs

In [50]:
reload(gliderpairs)
max_time = 10.
gPairs = gliderpairs.GliderPairs(arcterx2, max_time=max_time)

# Structure function

In [51]:
iz = 1
nbins = 20
rbins = 10**np.linspace(0., np.log10(400), nbins) # km
#
variables = 'dTdTdT'
# Velocity
gPairs.calc_delta(iz, variables, skip_velocity=True)
gPairs.calc_Sn(variables)

Sn_dict = gPairs.calc_Sn_vs_r(rbins, nboot=10000)

/home/xavier/Projects/Oceanography/python/cugn/cugn/gliderpairs.py:344: RuntimeWarning: Mean of empty slice
  avg_r.append(np.nanmean(self.r[in_r]))
/home/xavier/Projects/Oceanography/python/cugn/cugn/gliderpairs.py:345: RuntimeWarning: Mean of empty slice
  avg_S1.append(np.nanmean(self.S1[in_r]))
/home/xavier/miniconda3/envs/ocean/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/xavier/miniconda3/envs/ocean/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/xavier/Projects/Oceanography/python/cugn/cugn/gliderpairs.py:348: RuntimeWarning: Mean of empty slice
  avg_S2.append(np.nanmean(self.S2[in_r]))
/home/xavier/Projects/Oceanography/python/cugn/cugn/gliderpairs.py:350: RuntimeWarning: Mean of empty slice
  avg_S3.append(np.nanmean(self

In [52]:
Sn_dict

{'config': {'variables': 'dTdTdT',
  'N': array([  0,   0,   2,   7,  21,  33,  46,  59, 105, 130,  68, 101, 153,
          17,  46,   0,   0,   0,   0])},
 'r': array([        nan,         nan,  2.42961432,  3.38771461,  4.2118281 ,
         5.85009013,  7.63360585, 10.96634657, 14.91469472, 19.69588839,
        27.17662455, 38.51595553, 51.7824934 , 67.18998089, 98.43515396,
                nan,         nan,         nan,         nan]),
 'S1_dT': array([        nan,         nan,  0.33851812, -0.00309083,  0.00440638,
         0.01369238,  0.08498664,  0.04966812, -0.01698577, -0.00582641,
         0.00625205,  0.00288289,  0.04961013,  0.00988483, -0.05768436,
                nan,         nan,         nan,         nan]),
 'std_S1_dT': array([       nan,        nan, 0.31123284, 0.1703601 , 0.21644757,
        0.17463064, 0.21107875, 0.28446749, 0.22834877, 0.29417281,
        0.41160841, 0.56495959, 0.80114191, 0.74210463, 1.0462983 ,
               nan,        nan,        nan,        